# BERT

At todays meeting, we're going to focus on BERT -- a model, which transforms our tokens into contextual word embeddings, that are of great quality!

We will use `transformers` library for that purpose, so let's install it first:

In [ ]:
!pip install transformers


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


#BERT

## Task 1: Tokenization using WordPiece

Below you can find a simple fragment of code, which downloads a `bert-base` model (an `uncased` version -- this model introduced a preprocessing step on the data that transformed each text into lowercased text) and instantiates appropriate tokenizer for that model. You can learn about this particular model here: https://huggingface.co/bert-base-uncased (I encourage you to read the description! Many models hosted on the huggingface website have great documentations and it's always worth checking them).

Then, in line 4, we define a text to be tokenized, run the tokenizer in line 5, and use the tokenized input as the input to BERT model, which is called in line 6.

Please, run the code below. We generated BERT embeddings using 6 lines of code! ;)

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Runnning tokenizer by simply calling the tokenizer object (`tokenizer(text, return_tensors='pt')`) returns a dictionary containing 3 keys: `input_ids`, `token_type_ids`, and `attention_mask`. As for now, let's focus on `input_ids` only.

You may visit the website: https://huggingface.co/docs/transformers/glossary to learn more about the role of `token_type_ids` and `attention_mask`.


The `input_ids` is a list of lists (represented as a tensor). The outer list collects documents, while the inner lists collect tokens in that document. Here, we processed only one document (sentence) so that there is only one "outer" list.

Each of the inner lists contains a seqeunce of identifiers. These are the positions of tokens in the Vocabulary. They can be used to generate one-hot encoding representations for our words, since if we know the length of a vocabulary and know the position of a given token in the vocabulary, we can generate a vector of the vocabulary length that is filled with zeros, then we set the value assigned to a token position to 1 to generate one-hot encoding.

To produce the ids, we have to tokenize our text first.

Moreover, those identifiers require less memory that storing tokens as strings!

Run the code below, to see the tokenizer's output. Please note that the number of token identifiers generated by the tokenizer is not equal to the number of tokens in the sequence. We'll see why is that in a minute.


In [ ]:
encoded_input

{'input_ids': tensor([[ 101, 5672, 2033, 2011, 2151, 3793, 2017, 1005, 1040, 2066, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
print(len(text.split()), len(encoded_input['input_ids'][0]))

7 12


The ids generated by the tokenizer are not human readable. However, the tokenizer contains a mapping which relates known tokens to their positions, so that we can use it to map ids to tokens back!

The first line of code obtains the input ids defined for the first sequence provided.
Then, we call `convert_ids_to_tokens` to transform those ids to tokens. Please run the code and analyze the output.

In [ ]:
first_sentence_ids = encoded_input['input_ids'][0]
tokenizer.convert_ids_to_tokens(first_sentence_ids)

['[CLS]',
 'replace',
 'me',
 'by',
 'any',
 'text',
 'you',
 "'",
 'd',
 'like',
 '.',
 '[SEP]']

Whoa, we see that the tokenizer not only tokenizes our text (splits into tokens), but it also generated special tokens required by our model! Nice. Please note that after tokenization we don't have capital letters in our tokens. This is caused by the use of the `bert-base-uncased` model. Since the model was trained on lowercased data, the tokenizer also ensures that the tokens are lowercased.


## Subword units

However, the length of the generated `input_ids` may be even bigger as related to the length of our text. Sometimes, the vocabulary doesn't contain a given word as a whole. Since BERT used WordPiece tokenization, to handle such cases, the tokenizer tries to split those words into smaller fragments, that are stored in our vocabulary. Let's tokenize some document containing rare words and see the result.

In [ ]:
tokenizer_output = tokenizer(['NVIDIA DGX A100'])

input_ids = tokenizer_output['input_ids'][0]
input_ids

[101, 1050, 17258, 2401, 1040, 2290, 2595, 17350, 8889, 102]

Whoa, we see that as a result of tokenization, we obtained many tokens. Let's see what they represent:

In [ ]:
tokenizer.convert_ids_to_tokens(input_ids)

['[CLS]', 'n', '##vid', '##ia', 'd', '##g', '##x', 'a1', '##00', '[SEP]']

Ah, as we can see, the vocabulary assigned to `bert-base` doesn't contain tokens such as "nvidia", "dgx" and "a100", that's why they are split into subwrod units.

Each time a given subword unit starts with a double hash (##), we know that this subword unit is a continuation of the previous token.

We can use this information to reconstruct the original text, by joining those subword units (sometimes called subtokens) into full tokens. We can achieve this goal using the following line of code:

In [ ]:
tokenizer.decode(input_ids)

2023-11-02 10:39:41.951895: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-02 10:39:42.237543: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-02 10:39:43.357245: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


'[CLS] nvidia dgx a100 [SEP]'

If we know how to map tokens to their positions in the vocabulary, the only missing part is to determine how long our one-hot-encoding vector should be (or how big is our vocabulary).

This transformation of ids into one-hot-encodings is done automatically by the `transformer` library. However, you can check the size of the vocabulary easily using the following line of code:

In [ ]:
tokenizer.vocab_size

30522

## Task 2: Using a pre-trained BERT to generate features that may be used to solve a classification task.

As we discussed during the lecture, we can generate a fixed-length vector represening any input by taking the embedding produced for the `[CLS]` token (which is a representation of the whole seqeunce).

In this task, your goal is to use a pre-trained BERT model to obtain representations for a given dataset. Then, these representations will be used to train a logistic regression model.

We will try to genearate a solution that can detect whether a given review is positive or not!

For that purpose we're going to use a BERT variant called `distilBERT`. We didn't cover it during the lecture, because it is related to the concept of distillation that will be very important to us and I'll make a separate lecture on it.

For now, we can treat `distilBERT` the same way as BERT. In fact, this is a compressed BERT model. It behaves the same way but it is distilled so that we aimed to achieve similar quality using less parameters.

Please follow the tutorial you can find here: https://colab.research.google.com/github/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb (there is even a blogpost related to this tutorial: http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/)


Just copy and paste fragments of code here and observe the results:

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [ ]:
batch_1 = df[:2000] # cut dataset for performance reasons
batch_1[1].value_counts()

1    1041
0     959
Name: 1, dtype: int64

In [ ]:
# load pretrained distilbert
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model_class, tokenizer_class

(transformers.models.distilbert.modeling_distilbert.DistilBertModel,
 transformers.models.distilbert.tokenization_distilbert.DistilBertTokenizer)

In [ ]:
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

# pad to the same size so we can use them as batch
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

np.array(padded).shape

(2000, 59)

In [ ]:
# say bert that we have padded tokens
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(2000, 59)

In [ ]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
# get embedding for [CLS] token, representing the whole sentence
features = last_hidden_states[0][:,0,:].numpy()
labels = batch_1[1] # positive or negative sentiment

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

best parameters:  {'C': 5.263252631578947}
best scrores:  0.8033333333333333


In [ ]:
# train logistic regression on top of bert embeddings

lr_clf = LogisticRegression(C=5.263157894736842)
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=5.263157894736842)

In [ ]:
lr_clf.score(test_features, test_labels)

0.836

In [ ]:
# get dummy classifier score as a baseline

from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.521 (+/- 0.00)


## Task 3(Optional - not required): fine-tuning BERT

In the example above, a BERT model (or more precisely it's smaller family member: distilBERT) was used only to deliver vectors representing whole documents.

Now we would like to fine-tune an existing model. As we discussed during the lecture, we can achieve it by simply switching the top-layer of the network. Instead of solving the Masked Language Model and Next Sentence Prediction tasks, we may add our own classification layer (also referred to as a classification head) and train our whole network to solve a given task.

There is a great and easy to follow tutorial on fine-tuning, available here: https://github.com/huggingface/notebooks/blob/main/transformers_doc/training.ipynb

If you want, please follow that tutorial (you can copy-and-paste the code snippets from the notebook here).

In [ ]:
from IPython.display import HTML
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
from datasets import load_from_disk

try:
    !tar -xzvf ./tokenized_dataset.tar.gz
    tokenized_dataset = load_from_disk('./tokenized_dataset')
except:
    tokenized_dataset = dataset.map(lambda x: tokenizer(x['text'], truncation=True, padding="max_length"), batched=True)
    tokenized_dataset.save_to_disk('./tokenized_dataset')
    !tar -czvf ./tokenized_dataset.tar.gz ./tokenized_dataset

In [5]:
# get smaller dataset for performance reasons
small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(1000))

In [6]:
### easy – Trainer class from transformers

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5) # 5 is num of classes in this dataset

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# hyperparameters and training options

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch") # dir to save checkpoints and monitoring results each epoch

In [8]:
# track accuracy during training

import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-8-66a42e67f145>:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [10]:
# train
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.100781,0.572000
2,No log,1.131317,0.574000
3,No log,1.216907,0.586000


TrainOutput(global_step=375, training_loss=0.6678494059244792, metrics={'train_runtime': 408.8367, 'train_samples_per_second': 7.338, 'train_steps_per_second': 0.917, 'total_flos': 789354427392000.0, 'train_loss': 0.6678494059244792, 'epoch': 3.0})

In [57]:
one_example_dataset = small_eval_dataset.select(range(1))
predictions = trainer.predict(one_example_dataset)
print(f'{one_example_dataset[0]["text"]}\ntrue_label: {one_example_dataset[0]["label"]}\npredicted label: {predictions[1][0]}')


Kabuto is your run-of-the-mill Japanese Steakhouse. Different stations with chefs slinging shrimp tails around the communal dining areas like it's a lunchtime magic show. Always a plethora of laughs and gags going around the group. \n\nThis place is great for lunch. $9 and 30 minutes and you're out the door. Uhhh...If I'm craving a salad with ginger dressing, which I always am, (you do too. admit it) fried rice, steak, shrimp and white sauce (DUDE) then Kabuto is king of lunch options in my book. Always super clean and full of kindhearted staff. The parking lot is super difficult to get in and out of though. 51 traffic at lunch is a beast. Good luck getting stuck behind someone trying to cut across traffic at 12pm on a weekday. It's murder. This place would greatly benefit from another exit/entrance or a stoplight. Here's hoping....\n\nYou can't really shake a stick at balanced lunch when you can have soup or a salad, veggies, fried rice and a choice of a protein for under $10 and be b

In [ ]:
print(small_train_dataset[100].keys())

dict_keys(['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'])


In [ ]:
### KERAS
from transformers import DefaultDataCollator

# collator converts data to tf tensors and batch them
data_collator = DefaultDataCollator(return_tensors="tf")

# convert do tf dataset
tf_train_dataset = small_train_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = small_eval_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

# create a model from pretrained bert and train it
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
125/125 [==============================] - 215s 1s/step - loss: 1.5734 - sparse_categorical_accuracy: 0.2570 - val_loss: 1.3903 - val_sparse_categorical_accuracy: 0.3360
Epoch 2/3
125/125 [==============================] - 201s 2s/step - loss: 1.2290 - sparse_categorical_accuracy: 0.4580 - val_loss: 1.2083 - val_sparse_categorical_accuracy: 0.4880
Epoch 3/3
125/125 [==============================] - 162s 1s/step - loss: 0.9791 - sparse_categorical_accuracy: 0.5680 - val_loss: 1.0403 - val_sparse_categorical_accuracy: 0.5460


In [ ]:
# free some memory
del model
del trainer
torch.cuda.empty_cache()

In [67]:
### PYTORCH
from torch.utils.data import DataLoader

# prepare dataset
tokenized_datasets_torch = tokenized_dataset.remove_columns(["text"]) # model doesn't accept raw text
tokenized_datasets_torch = tokenized_datasets_torch.rename_column("label", "labels") # model expects "labels"
tokenized_datasets_torch.set_format("torch") # convert to pytorch tensors

# get smaller piece
small_train_dataset = tokenized_datasets_torch["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets_torch["test"].shuffle(seed=42).select(range(1000))

# to iterate over batches
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)


In [62]:
from transformers import AutoModelForSequenceClassification # not tf this time
import torch
from torch.optim import AdamW
from transformers import get_scheduler

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)
optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

# move model to GPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [71]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

metric = load_metric("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

  0%|          | 0/375 [00:00<?, ?it/s]

{'accuracy': 0.571}

In [96]:
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    break
logits = outputs.logits
predictions = torch.argmax(logits, dim=-1)

example = small_eval_dataset.select(range(1))[0]
print(f'{tokenizer.decode(example["input_ids"])}\ntrue label: {example["labels"]}\npredicted label: {predictions[0]}')


[CLS] Kabuto is your run - of - the - mill Japanese Steakhouse. Different stations with chefs slinging shrimp tails around the communal dining areas like it's a lunchtime magic show. Always a plethora of laughs and gags going around the group. \ n \ nThis place is great for lunch. $ 9 and 30 minutes and you're out the door. Uhhh... If I'm craving a salad with ginger dressing, which I always am, ( you do too. admit it ) fried rice, steak, shrimp and white sauce ( DUDE ) then Kabuto is king of lunch options in my book. Always super clean and full of kindhearted staff. The parking lot is super difficult to get in and out of though. 51 traffic at lunch is a beast. Good luck getting stuck behind someone trying to cut across traffic at 12pm on a weekday. It's murder. This place would greatly benefit from another exit / entrance or a stoplight. Here's hoping.... \ n \ nYou can't really shake a stick at balanced lunch when you can have soup or a salad, veggies, fried rice and a choice of a pro